# In this Notebook we are attempting to predict if a person will win their next fight depending on how they performed on their last 5 fights

## Library Imports

In [ ]:
import sys
sys.path.insert(1, '../combined_data')
sys.path.insert(1, '../predict_winner')
from make_career import make_career
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import talos as ta
from talos.model.normalizers import lr_normalizer
from talos.model.hidden_layers import hidden_layers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

## Loading the dataset

In [ ]:
features, labels = make_career(pd.read_csv('../combined_data/combined_fight_data_zerod_nans.csv'))
labels = labels.reshape(-1,)
labels = LabelEncoder().fit_transform(labels)

In [ ]:
print(features.shape)
print(type(features))

In [ ]:
print(labels.shape)
print(type(labels))

In [ ]:
print(labels)

## Preprocess Features

## Generates np.array of shape (3220, 5 * features)

In [ ]:
def collapse_n_fights():
    num_cols, cat_cols = get_column_types()
    
    # Collect only the features we want
    new_features = []
    for window in features:
        window_arr = list(window.reshape(1,-1))[0]
        fight_window = np.array([])
        for fight in window_arr:
            fight_arr = np.array([fight[i] for i in num_cols])
            np.nan_to_num(fight_arr, copy=False)
            fight_window = np.append(fight_window, fight_arr)
        new_features.append(fight_window)
    
    new_features = np.array(new_features)
    X_train, X_test, y_train, y_test = train_test_split(new_features, labels, random_state=0, train_size=0.92)
    
    # Scale the data
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    print("X_train length {}, y_train length {}".format(X_train.shape, y_train.shape))
    print("X_test length {}, y_test length {}".format(X_test.shape, y_test.shape))
    
    return X_train, X_test, y_train, y_test

## Generate np.array of shape (3220, 5, features)

In [ ]:
def n_fights_to_array():
    num_cols, cat_cols = get_column_types()
    
    new_features = []
    for window in features:
        window_arr = list(window.reshape(1,-1))[0]
        fight_window = []
        for fight in window_arr:
            fight_arr = np.array([fight[i] for i in num_cols])
            np.nan_to_num(fight_arr, copy=False)
            fight_window.append(fight_arr)
        new_features.append(fight_window)
    
    new_features = np.array(new_features)
    X_train, X_test, y_train, y_test = train_test_split(new_features, labels, random_state=0, train_size=0.80)
    print("X_train length {}, y_train length {}".format(X_train.shape, y_train.shape))
    print("X_test length {}, y_test length {}".format(X_test.shape, y_test.shape))

    return X_train, X_test, y_train, y_test

## Building a DNN to predict the winner using last n fights

In [ ]:
# Get the data
X_train, X_test, y_train, y_test = collapse_n_fights()

In [ ]:
# Define hyperparameters to use in Grid Search
dnn_params = {'lr': (0.2, 1, 2),
     'first_neuron': [128, 256],
     'hidden_layers': [1, 2],
     'batch_size': [64, 128, 256],
     'epochs': [10, 25],
     'dropout': (0, 0.5, 3),
     'optimizer': [Adam],
     'shapes':['funnel'],
     'losses': [binary_crossentropy],
     'activation': [relu],
     'last_activation': [sigmoid]}

In [ ]:
# Create the Neural Network
dnn_model = ta.Scan(x=X_train,
               y=y_train,
               model=get_dnn,
               params=dnn_params,
               experiment_name="UFC_5_Fight_Predictor")

In [449]:
results_df = dnn_model.data

In [450]:
top_10_dnn = results_df.sort_values(by=['val_accuracy'], ascending=False).head(10)

In [451]:
top_10_dnn

,round_epochs,val_loss,val_accuracy,loss,accuracy,activation,batch_size,dropout,epochs,first_neuron,hidden_layers,last_activation,losses,lr,optimizer,shapes
119,10,0.696134,0.596378,0.678927,0.624515,<function relu at 0x1370a5d90>,256,0.166667,10,256,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.6,<class 'keras.optimizers.Adam'>,funnel
7,10,0.737508,0.593790,0.602896,0.707155,<function relu at 0x1370a5d90>,64,0.000000,10,256,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.6,<class 'keras.optimizers.Adam'>,funnel
10,25,0.778712,0.591203,0.430949,0.829174,<function relu at 0x1370a5d90>,64,0.000000,25,128,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,<class 'keras.optimizers.Adam'>,funnel
3,10,0.747239,0.589909,0.492866,0.774820,<function relu at 0x1370a5d90>,64,0.000000,10,128,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.6,<class 'keras.optimizers.Adam'>,funnel
26,25,0.720206,0.588616,0.559412,0.687743,<function relu at 0x1370a5d90>,64,0.166667,25,128,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,<class 'keras.optimizers.Adam'>,funnel
56,25,0.719491,0.587322,0.631799,0.670549,<function relu at 0x1370a5d90>,128,0.000000,25,128,1,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,<class 'keras.optimizers.Adam'>,funnel
14,25,0.871737,0.587322,0.359246,0.899612,<function relu at 0x1370a5d90>,64,0.000000,25,256,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,<class 'keras.optimizers.Adam'>,funnel
110,25,0.734341,0.586028,0.433027,0.824737,<function relu at 0x1370a5d90>,256,0.000000,25,256,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,<class 'keras.optimizers.Adam'>,funnel
126,25,0.702003,0.582147,0.683113,0.616195,<function relu at 0x1370a5d90>,256,0.166667,25,256,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,<class 'keras.optimizers.Adam'>,funnel
69,10,0.701525,0.580854,0.671244,0.620078,<function relu at 0x1370a5d90>,128,0.166667,10,256,1,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.6,<class 'keras.optimizers.Adam'>,funnel


In [452]:
top_10_dnn.to_csv("../model_results/UFC_Predict_5_Fights_DNN.csv")

## Building a CNN to predict the winner using last n fights

In [ ]:
# Get the data
X_train, X_test, y_train, y_test = collapse_n_fights()
X_train = X_train.reshape(X_train.shape[0], 5, 85)
X_test = X_test.reshape(X_test.shape[0], 5, 85)

print("New: X_train length {}, y_train length {}".format(X_train.shape, y_train.shape))
print("New: X_test length {}, y_test length {}".format(X_test.shape, y_test.shape))

In [ ]:
# Define hyperparameters to use in Grid Search
cnn_params = {'lr': (0.2, 1, 2),
     'num_filters': [64, 128],
     'kernel_size': [1, 2, 3],
     'batch_size': [64, 128],
     'epochs': [5, 10, 15],
     'dropout': (0, 0.5, 2),
     'flatten_layer': [100, 150], 
     'optimizer': [Adam],
     'losses': [binary_crossentropy],
     'activation': [relu],
     'last_activation': [sigmoid]}

In [ ]:
# Create the Neural Network
cnn_model = ta.Scan(x=X_train,
                y=y_train,
                model=get_cnn,
                params=cnn_params,
                experiment_name="UFC_5_Fight_Predictor_CNN")

In [445]:
results_df = cnn_model.data

In [446]:
top_10_cnn = results_df.sort_values(by=['val_accuracy'], ascending=False).head(10)

In [447]:
top_10_cnn

,round_epochs,val_loss,val_accuracy,loss,accuracy,activation,batch_size,dropout,epochs,flatten_layer,kernel_size,last_activation,losses,lr,num_filters,optimizer
84,5,0.682176,0.601552,0.657059,0.617859,<function relu at 0x1370a5d90>,64,0.25,5,150,1,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,64,<class 'keras.optimizers.Adam'>
218,5,0.685183,0.596378,0.638778,0.620632,<function relu at 0x1370a5d90>,128,0.25,5,100,1,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.6,64,<class 'keras.optimizers.Adam'>
268,15,0.682486,0.593790,0.627070,0.642263,<function relu at 0x1370a5d90>,128,0.25,15,100,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,64,<class 'keras.optimizers.Adam'>
272,15,0.681047,0.593790,0.618065,0.671104,<function relu at 0x1370a5d90>,128,0.25,15,100,3,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,64,<class 'keras.optimizers.Adam'>
108,10,0.685162,0.591203,0.627414,0.648364,<function relu at 0x1370a5d90>,64,0.25,10,150,1,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,64,<class 'keras.optimizers.Adam'>
164,5,0.677643,0.589909,0.653327,0.622296,<function relu at 0x1370a5d90>,128,0.00,5,150,3,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,64,<class 'keras.optimizers.Adam'>
80,5,0.676952,0.588616,0.654303,0.612867,<function relu at 0x1370a5d90>,64,0.25,5,100,3,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,64,<class 'keras.optimizers.Adam'>
145,5,0.683330,0.588616,0.622050,0.672768,<function relu at 0x1370a5d90>,128,0.00,5,100,1,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,128,<class 'keras.optimizers.Adam'>
185,10,0.687761,0.588616,0.574967,0.732668,<function relu at 0x1370a5d90>,128,0.00,10,150,2,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.2,128,<class 'keras.optimizers.Adam'>
219,5,0.689400,0.588616,0.618474,0.660011,<function relu at 0x1370a5d90>,128,0.25,5,100,1,<function sigmoid at 0x1370a5ea0>,<function binary_crossentropy at 0x137054730>,0.6,128,<class 'keras.optimizers.Adam'>


In [448]:
top_10_cnn.to_csv("../model_results/UFC_Predict_5_Fights_CNN.csv")

## Building an LSTM to predict the winner using last n fights

# Model Builders

In [ ]:
# Model imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
from keras.optimizers import Adam, Nadam, RMSprop
from keras.losses import logcosh, binary_crossentropy
from keras.activations import relu, elu, sigmoid

### Analyzing last n fights using a Dense Neural Network

In [ ]:
def get_dnn(x_train, y_train, x_val, y_val, params, test_model=False):
    
    model = Sequential()
    # Input Layer
    model.add(Dense(params["first_neuron"], 
                    activation=params['activation'], 
                    input_dim=X_train.shape[1]))
    
    model.add(Dropout(params['dropout']))
    
    # Hidden Layers
    hidden_layers(model, params, 1)
    
    # Output Layers
    model.add(Dense(1, activation=params['last_activation']))
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])), 
                  metrics=['accuracy'])
                  
    history = model.fit(x_train, y_train,
                       validation_data=[x_val, y_val],
                       batch_size=params['batch_size'],
                       epochs=params['epochs'],
                       verbose=0)
    
    if test_model:
        model.summary()
        plot_training_results(history)
        

    return history, model

### Analyzing last n fights using a Convolutional Neural Network

In [ ]:
def get_cnn(x_train, y_train, x_val, y_val, params, test_model=False):
    model = models.Sequential()
    
    # Convolutional Layers
    model.add(Conv1D(filters=params["num_filters"], kernel_size=params["kernel_size"], activation='relu', input_shape=(5, 85)))
    model.add(Conv1D(filters=params["num_filters"], kernel_size=params["kernel_size"], activation='relu'))
    model.add(Dropout(params['dropout']))
#     model.add(MaxPooling1D(pool_size=2))
    
    # Flatten Layers
    model.add(Flatten())
    model.add(Dense(params["flatten_layer"], activation='relu'))
#     model.add(Dense(params["flatten_layer"], activation='relu'))
    
    # Output Layer
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])), 
                  metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,
                       validation_data=[x_val, y_val],
                       batch_size=params['batch_size'],
                       epochs=params['epochs'],
                       verbose=1)
    
    if test_model:
        plot_training_results(history)
        model.summary()

    return history, model

## Helper Functions

In [ ]:
def plot_training_results(history_obj):
    acc = history_obj.history['accuracy']
    val_acc = history_obj.history['val_accuracy']
    loss = history_obj.history['loss']
    val_loss = history_obj.history['val_loss']
    
    epochs = range(1, len(acc) + 1)

    fig, (accuracy_axis, loss_axis) = plt.subplots(1, 2, figsize=(20, 5))

    accuracy_axis.plot(epochs, acc, 'bo', label='Training acc')
    accuracy_axis.plot(epochs, val_acc, 'b', label='Validation acc')
    accuracy_axis.set_title('Training and validation accuracy')
    accuracy_axis.set_ylabel('Accuracy')
    accuracy_axis.set_xlabel('Epoch')
    accuracy_axis.legend()

    loss_axis.plot(epochs, loss, 'bo', label='Training loss')
    loss_axis.plot(epochs, val_loss, 'b', label='Validation loss')
    loss_axis.set_title('Training and validation loss')
    loss_axis.set_ylim(0, 3)
    loss_axis.set_ylabel('Loss')
    loss_axis.set_xlabel('Epoch')
    loss_axis.legend()
    fig.show()

## Extract numerical and categorical columns

In [ ]:
# Get the first fight window and the first fight in that window
def get_column_types():
    num_cols = []
    cat_cols = []
    for fight in features[0][0]:
        feature_type = type(features[1][1][fight])
        if feature_type is not float and feature_type is not int:
            cat_cols.append(fight)
        else:
            num_cols.append(fight)
            
    return num_cols, cat_cols

In [ ]:
x, y = get_column_types()

In [ ]:
y